### 数据类

tuple的子类
- colllections.namedtuple
- typing.NameTuple
装饰器类
- @dataclass
> 这些都会提供必要的方法__init__,__repr__,__eq__等方法 

### 主要功能
1. 可变实例
    collecions.namedtuple和typing.NamedTuple构建的类时tuple的子类,因此实例是不可变的.@dataclass默认构建的是可变类,可以用frozen参数来控制
2. class语句语法
    typing.NamedTuple和@dataclass支持常规class语句
3. 构造字典
    两个具名元组都提供了构造dict对象的方法(._asdict),可根据数据类实例字段构造字典;dataclasses模块也提供了构造字典的函数dataclass.asdict
4. 获取字段名称和默认值
    3个类构建器都支持获取字段名称和可能配置的默认值,具名元组类原数据在._fields和._fields_defaults中.对于使用dataclass修饰的类,则使用dataclasses模块中fields函数获取,fields函数返回一个由Field对象构成的元组
5. 获取字段类型
    typing.NamedTuple和@dataclass定义的类有一个__annotations__类属性,最好不要直接获取,可使用typing.get_type_hints函数
6. 更改之后创建新的实例
    对于具名元组实例x,可以用x._replace(**kwargs)根据指定关键字参数替换某些属性值返回一个新实例.dataclass修饰的类则用
    dataclasses.replace(x,**kwargs)
7. 运行时定义新类 
    collections.namedtuple和typing.NamedTuple可以在运行生成类,dataclasses模块也提供了make_dataclass方法来实现


In [ ]:
from collections import namedtuple
Coordinate=namedtuple('Coordinate','lat,lon')
c1=Coordinate(1,2)
print(c1)

from typing import NamedTuple
class TypeCoordinate(NamedTuple):
    lat:float
    lon:float

c2=TypeCoordinate(1.0,2.0)
print(c2)
print(issubclass(Coordinate,tuple),issubclass(TypeCoordinate,tuple))

from dataclasses import dataclass
@dataclass
class ClCoordinate():
    lat:float
    lon:float

c3=ClCoordinate(1.2,2.2)
print(c3)

@dataclass(frozen=True)
class FrozenCoordinate():
    lat:float
    lon:float
c4=FrozenCoordinate(1,2)
print(c4)
# c4.lat=12 @dataclass 添加frozen=True让对象不可变

### namedtuple额外方法
- ._fields:返回存储类的字段名称
- ._make(iterabel):根据可迭代对象构建实例
- ._asdict:将元组实例构造成dict对象

In [22]:
from collections import namedtuple
# namedtuple 接受defulats参数,值为一个可迭代对象,为从右开始计算
Coordinate=namedtuple('Coordinate','lat,lon,name',defaults=['coor'])

c1=Coordinate(1,2)

print('_asditc()',c1._asdict)
c2=Coordinate._make((3,3,2))
print(c2)
print('_field()',Coordinate._fields)
print('_field_default()',Coordinate._field_defaults)



_asditc() <bound method Coordinate._asdict of Coordinate(lat=1, lon=2, name='coor')>
Coordinate(lat=3, lon=3, name=2)
_field() ('lat', 'lon', 'name')
_field_default() {'name': 'coor'}


### 类型提示
类型提示也叫类型注解,声明函数参数,返回值,变量和属性的期望类型.python字节码编译器和解释器根本不强制提供类型信息.也就是运行时不检查类型,类型主要为第三方类型检查工具提供支持,mypy和ide内置的额类型检查器
`mypy test.py`

**注解语法**
在class语句中定义语法
    > var: type
容器化类型
    > list[str],tuple[str,float]
可选类型
    > typing.Optional,Optional[str],申明一个字段类型可以是str或None

**注解的意义**
类型提示在运行时没有用,在python导入模块会读取类型,构建__annotation__字典,供其他工具使用

In [10]:
class DemoClass:
    a: int # a 被放弃因为类没有为a名称的属性
    b: float = 1.1 # b是一个类属性,值为1.1
    c = 'span' # c是普通类属性没注解

print(DemoClass.__annotations__)

from typing import NamedTuple
class DemoNdClass(NamedTuple):
    a:int # a是注解也是实例属性
    b:float=1.1 # b是注解也是实例属性,默认值1.1
    c='span'  # 普通类属性,没有注解

print(DemoNdClass.a)
print(DemoNdClass.c)
d3=DemoNdClass(2)
# 类里面的值是只读属性,不能进行赋值,因为实例毕竟时元组

from dataclasses import dataclass
@dataclass
class DemoDataClass:
    a: int # 注解,实例属性
    b: float = 1.1 # 注解,实例属性,默认值为1.1 
    c = 'span' # 普通类属性,没注解

d4=DemoDataClass(1)
d4.c='c'
print(d4.c,DemoDataClass.c)

{'a': <class 'int'>, 'b': <class 'float'>}
_tuplegetter(0, 'Alias for field number 0')
span
c span


### @dataclass
`@dataclass(*,init=True,repr=True,eq=True,order=False,unsafe_hash=False,frozen=False)`

- init:生成__init__方法,如果自己实现,则忽略该参数
- repr:生成__repr__方法,
- eq:生成__eq__方法,
- order:生成__lt__,__le__,__gt__,__ge__,如果设置为True,eq=false自行定义或继承其他用于比较的方法,则抛出异常
- unsafe_hash:生成__hash__
- frozen:防止更改实例

> 如果eq和frozen都为True,那么@dataclass将生成一个合适的__hash__方法,确保实例是可哈希;反之frozen为false,@dataclass把__hash__设置为None,覆盖从任何超类继承的__hash__方法.

> **python规定带默认值的参数后面不能有不带默认值的参数,因此为一个字段声明默认值后,余下的字段都要有默认值**

### field
为定义的数据类添加默认参数
- default:字段的默认值;_MISSING_TYPE
- default_factory:不接受参数的函数,用于产生默认值;_MISSING_TYPE
- init:把字段作为参数传给__init__,True
- repr:在__repr__中使用字段,True
- compare:在__eq__,__lt__等比较方法中使用字段,True
- hash:在__hash__方法中使用字段计算hash,None
- metadate:用户定义的数据映射,@dataclass忽略该参数,None

### 初始化后处理
在__init__方法只做一件事,把传入的参数及默认值赋值给实例属性,在调用方后最后还调用__post_init__

### 带类型的类属性
使用typing.ClassVar伪类型


### 初始化不作为字段的变量
使用typing.InitVar阻止把字段作为常规字段 

In [15]:
from dataclasses import dataclass,field

@dataclass
class Member:
    name:str
    guests:list=field(default_factory=list,init=False)
    vip:bool=field(default=False,init=False)

    def __post_init__(self):
        self.guests.append('13')

m1=Member('21')
m1.guests.append(1)
print(m1.guests)


from typing import ClassVar
from dataclasses import InitVar

@dataclass
class VarClass:
    out:ClassVar[str]='out'
    init:InitVar[list[str]]=list()

var=VarClass()
print(var.init)
print(VarClass.out)

['13', 1]
[]
out
